In [ ]:
%load_ext autoreload
%autoreload 2

import sys

In [ ]:
import os
import array
import re

import pandas
import numpy

import plotly.express
import librosa
from matplotlib import pyplot as plt

from software.features.spectral import design_bandpass, plot_bandpass
from software.features.featureutils import resample, rms, vector_magnitude
from software.dataset.combine import load_data, read_labels, parse_video_filename, apply_labels

In [ ]:
combined_path = 'data/jonnor-brushing-1/combined.parquet'

samplerate = 50
lower = 2.0
upper = 5.0

In [ ]:
data = pandas.read_parquet(combined_path)
data

In [ ]:
import tempfile
import subprocess

def run_system(data, micropython_bin='micropython'):

    script_path = './firmware/process.py'
    
    with tempfile.TemporaryDirectory() as tmpdir:
        data_path = os.path.join(tmpdir, 'input.npy')
        out_path = os.path.join(tmpdir, 'out.csv')

        # write input data to file
        arr = numpy.ascontiguousarray(data) # make sure its C order
        numpy.save(data_path, arr, allow_pickle=True)
    
        args = [
            micropython_bin,
            script_path,
            data_path,
            out_path,
        ]
        stdout = subprocess.check_output(args).decode('utf-8')

        # read output file
        assert os.path.exists(out_path), 'script did not write output'
        out = pandas.read_csv(out_path)
        return out, stdout
        

def plot_session(sensors, labels=None, title=None, height=300, aspect=3.0):

    time_column = 'time'
    columns = [
        #'x', 'y', 'z',
        'is_brushing'
    ]
    width = height * aspect
    annotation_column = 'class'
    
    # TODO: plot other values than raw XYZ.
    # Like motion, from removing gravity. Maybe overall RMS, delta rms, band-passed 2-5hz instead of  
    sel = sensors.reset_index()
    fig = plotly.express.line(sel,
                              x=time_column,
                              y=columns,
                              width=width,
                              height=height,
                              title=title,
                             )
    fig.update_traces(connectgaps=False)
    fig.update_layout(showlegend=False)


    # FIXME: put this on the bottom
    if labels is not None:
        colors = plotly.express.colors.qualitative.Plotly
        types = labels[annotation_column].unique()
        annotation_colors = { v: c for v, c in zip(types, colors) }
        
        for idx, l in labels.iterrows():
            #print(dict(l))
            color = annotation_colors[l[annotation_column]]
            fig.add_vrect(x0=l['start_time'],
                    x1=l['end_time'],
                    line_width=0,
                    fillcolor=color,
                    opacity=0.3,
                    label=dict(
                        text=l[annotation_column],
                        textposition="start",
                        font=dict(size=10, color="black"),
                        yanchor="top",
                    ),
             )
    
    return fig
    

data_columns = ['x', 'y', 'z']
sensitivity = 2.0
for session, session_data in data.groupby('session'):

    session_data['is_brushing'] = session_data['is_brushing'].astype(int).astype(float)
    d = session_data.sort_index().reset_index()[data_columns]
    p =  f'{session}.npy'
    scaled = ((d.values / sensitivity) * (2**15-1)).astype(numpy.int16)

    out, stdout = run_system(scaled)
    out['t'] = pandas.to_timedelta(out['time'], unit='s')
    start_time = session_data.index.min()
    print(type(start_time), start_time)
    out['time'] = out['t'] + start_time

    fig = plot_session(session_data)

    from plotly.graph_objects import Scatter
    fig.add_trace(Scatter(x=out['time'],
                             y=out['brushing'],
                        mode='lines',
                        name='brushing_predicted'))
    
    fig.show()
    
    brush_time_total = out['brushing_time'].max()
    print(session, brush_time_total)


In [ ]:

import cv2

def overlay_video(video_path, out_path, timeline, display=False):
    
    assert os.path.exists(video_path), video_path 
    cap = cv2.VideoCapture(video_path)
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))   # float `width`
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # float `height`
    print('open-video', fps, width, height)

    format = cv2.VideoWriter_fourcc(*'MPEG')
    output = cv2.VideoWriter(out_path, format, fps, (width,height))

    # FIXME: the timing seems to be off??
    time = 0.0
    while cap.isOpened():
        ret, frame = cap.read()
    
        if not ret:
            print('no frame read')
            break

        # find predicted brushing state
        m = timeline[timeline.time >= time].iloc[0]
        is_brushing = m.brushing > 0.6
        #print(m)
        
        overlay = numpy.zeros_like(frame, np.uint8)
        out = frame.copy()

        # TODO: also render brushing_time
        # TODO: prettify a bit
        if is_brushing:
            # Render text
    
            # Add a semi-opaque box as background
            pad_left = 10 
            pad_right = 10
            rect_height = 100
            rect_y = height - 100
            start_point = (pad_left, rect_y)
            end_point = (width - pad_right, rect_y-rect_height)
            bg_color = (255, 255, 255)
            thickness = -1
            overlay_alpha = 0.5
            cv2.rectangle(overlay, start_point, end_point, bg_color, thickness)
    
            mask = overlay.astype(bool)
            out[mask] = cv2.addWeighted(frame, overlay_alpha, overlay, 1 - overlay_alpha, 0)[mask]
    
            # Render text
            font = cv2.FONT_HERSHEY_SIMPLEX
            text_color = (255,255,255)
            text_thickness = 2
            scale = 2.0
            text_pos = (pad_left+20, rect_y-20)
            cv2.putText(out,'BRUSHING', text_pos, font, scale, text_color, text_thickness)

        # TODO: render emlearn logo
        
        output.write(out)
        
        if display:
            cv2.imshow('frame', out)
            if cv2.waitKey(25) == ord('q'):
                break
            
        time += (1.0/fps)

    cap.release()
    cv.destroyAllWindows()
    print('done')



video_dir = '/home/jon/Downloads/toothbrush-jonnor1/edited/'

for session, session_data in data.groupby('session'):

    session_data['is_brushing'] = session_data['is_brushing'].astype(int).astype(float)
    d = session_data.sort_index().reset_index()[data_columns]
    p =  f'{session}.npy'
    scaled = ((d.values / sensitivity) * (2**15-1)).astype(numpy.int16)

    out, stdout = run_system(scaled)

    input_video = session
    out_path = 'overlay_'+input_video
    video_path = os.path.join(video_dir, input_video)

    print(input_video)
    overlay_video(video_path, out_path, out, display=False)

    break # XXX: TEMP





In [ ]:



for session, session_data in ss.groupby('session'):
    print(session)
    ll = mm.set_index('filename').loc[session]
    fig = plot_session(session_data, ll, title=session)
    fig.show()

In [ ]:
print(m.elapsed.max())